In [33]:
import os
from os.path import isfile, join
from torchtext.data import RawField, Field, TabularDataset, BucketIterator
import pandas as pd

def get_dir_files(dir_path):
    list_file = [f for f in os.listdir(dir_path) if isfile(join(dir_path, f))]
    return list_file

train_dataset_path = './dataset/processed_zinc_smiles/data_xs/train'
val_dataset_path = './dataset/processed_zinc_smiles/data_xs/val'

list_trains = get_dir_files(train_dataset_path)

# pd.read_csv(join(train_dataset_path, list_trains[0])).head(5)

In [36]:
SMILE = RawField()
logP = RawField()
MR = RawField()
TPSA = RawField()
LENGTH = RawField()

In [43]:
train_datafields = [("smile", SMILE),
                    ("logP", logP),
                    ("mr", MR), 
                    ("tpsa", TPSA),
                    ("length", LENGTH)]
train_dataset = TabularDataset(path=join(train_dataset_path, list_trains[0]),
                               format='csv',
                               skip_header=True,
                               fields=train_datafields)

In [44]:
train_dataset[1].logP
print(train_dataset[1].__dict__.keys())

dict_keys(['smile', 'logP', 'mr', 'tpsa', 'length'])


In [45]:
train_iter = BucketIterator(dataset=train_dataset, 
                            batch_size=3, 
                            sort_key=lambda x: len(x.length),
                            sort_within_batch=True, 
                            shuffle=True, 
                            sort=True)

In [46]:
next(train_iter.__iter__())


[torchtext.data.batch.Batch of size 3]
	[.smile]:['C[C@H]1C[C@](C)(N)C[C@@H]1O', 'Nc1nccc(Cl)c1I', 'N=CN[C@@H]1CCCNC1']
	[.logP]:['0.4946', '1.9218', '-0.06493']
	[.mr]:['36.9972', '46.3764', '37.8111']
	[.tpsa]:['46.25', '38.91', '47.91']
	[.length]:['9', '9', '9']